In [1]:
pip install diffusers transformers accelerate --upgrade --q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import zipfile
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import AutoPipelineForText2Image
import torch
from PIL import Image
from transformers import pipeline
import random
import shutil


2024-06-15 11:52:22.442302: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 11:52:22.442453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 11:52:22.574545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_

In [3]:
pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")

model_index.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

text_encoder_2/config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.29.0",
  "_name_or_path": "stabilityai/sdxl-turbo",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerAncestralDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [4]:
prompt = "orange phone claymorphism,3d render,icon"
image = pipe(prompt=prompt, num_inference_steps=5, guidance_scale=0.1).images[0]
image

  0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
category_matching = {
"Credit": ["TV", "Slates", "Lifeline", "Plane tickets", "Island", "Piggy bank", "Calculator", "Wallet", "Folder with documents", "Bank", "Stack of coins", "Percentage sign",
"Account", "House", "Sprout"
],
"Car loan": [
"Car", "Car keys", "Wheel", "Motorcycle", "Pack of coins", "Calculator", "Geolocation label",
"Journey", "Highway", "Percentage sign", "Wrench", "Bundle of money", "Car with a bow", "Gear", "Car with smoke from under the wheels", "Traffic light"
],
    "Accounts and deposits": [
"Glasses", "ATM", "Book", "Safe", "Wad of money",
"Money stack", "Piggy bank", "Wallet", "Table",
"Growth chart", "Phone", "Diogram",
"Schedule", "Percentage sign", "Sprout", "Pen"
        ],
"Insurance": [
"Shield sign", "Umbrella", "Tablet", "Calculator", "Helmet", "Document Folder",
"Heart Sign", "Ambulance", "Tree", "Sprout", "Office building", "Parachute", "First aid kit"
        ],
"Mortgage": [
"House", "Keys", "Dog", "Bank", "Stack of coins", "Construction crane",
"Schedule", "Moving boxes", "Stack of bills", "Tree", "Berry patch"
        ],
"Currency exchange": [
"ATM", "Banknotes", "Table", "Target", "Bow and Arrow", "Arrow",
"Circular histogram", "Histogram", "Travel", "Palm Tree", "Vacation", "Bank Building"
],
"Card": [
"Credit card", "Magnifying glass", "PIN code", "Stack of coins", "Phone", "Grocery cart", "Watch", "Shopping bags",
"Terminal"
]
}


In [5]:
objects = [
"a cat that protects savings",
"a house that grows in value",
"a tree with strong roots",
"a cup filled with reliability",
"a book full of investment advice",
"a pen writing financial strategies",
"a chair that supports in difficult times",
"a phone connected with global finance",
"a clock counting down interest",
"a computer that provides financial security",
"glasses that increase the visibility of income",
"a backpack full of possibilities",
"a car that provides fuel savings",
    "ATM that provides financial stability",
"card that opens the door to success",
"pillow that brings financial comfort",
    "a football that strengthens investments",
"a fireplace that warms financial investments",
"a store where every purchase is an investment",
"a hat that brings good luck in finance",
"a column that inspires financial decisions",
"cookies that come true investment forecasts",
    "pizza satisfying the financial appetite",
    "a chest of drawers storing valuable investments",
"a plaid in which it is comfortable with investments",
    "a lantern illuminating financial paths",
"a Rubik's cube assembled from economic strategies",
    "the wheel of fortune turning to financial success",
"the skating rink on which skating leads to wealth",
"the bus transporting to global finance",
"the submarine providing investment protection",
    "a sock that protects financial assets",
"a bowling ball that allows you to strike at financial success",
"a hammer that strengthens financial foundations",
"a doll that displays investment opportunities",
"a monocle that opens up a view of financial success",
    "chess, where every move is an investment strategy",
    "a fridge magnet that attaches financial goals",
"a heart rate monitor that tracks financial condition",
"a bouquet of flowers that attracts financial growth",
"a ring symbolizing financial stability",
"a fireplace in which investments are always burning",
"a skateboard that accelerates on the way to wealth",
"a concert ticket opening up access to financial benefits",
"a tennis ball maneuvering to economic success",
"a vacuum cleaner removing financial obstacles",
    "a skateboard that speeds up on the path to wealth"
]

In [11]:
objects = [
    "dragon",
    "dragon on a pile of money",
    "headphones",
    "podcast",
    "microphone",
    "heart cardiogram",
    "fish",
    "boat",
    "cloud",
    "airplane in the cloud",
    "matryoshka",
    "candy",
    "bear",
    "check mark",
    "thumbs up sign",
    "thumbs down sign",
    "hourglass",
    "key",
    "rocking horse",
    "to-do list"
]

In [12]:
colors = [
    'Red', 'Orange', 'Yellow', 'Green', 'Blue', 'Purple', 'Pink', 'Brown', 'Black', 'White',
    'Gray', 'Silver', 'Gold'
]

In [13]:
save_dir = "/kaggle/working/"
archive_count = 0

In [7]:
def generate_random_prompt(category, category_matching, colors):
    prompts_list = category_matching[category]
    k = random.randint(1, 3)
    random_objects = random.sample(prompts_list, k)
    # print(random_objects)
    colored_objects = []
    for obj in random_objects:
        color = random.choice(colors)
        colored_object = f"{color} {obj}".lower()
        colored_objects.append(colored_object)
    result_string = ', '.join(colored_objects)
    return result_string

In [8]:
def save_image_and_prompt(image, prompt, iteration):
    image_filename = os.path.join(save_dir, f"{iteration}.png")
    prompt_filename = os.path.join(save_dir, f"{iteration}.txt")
    
    image.save(image_filename)
    with open(prompt_filename, "w") as f:
        f.write(prompt)

In [9]:
def archive_files(iteration, archive_count):
    with zipfile.ZipFile(os.path.join(save_dir, f"archive_{archive_count}.zip"), 'w') as zipf:
        for i in range(iteration - 499, iteration + 1):
            img_file = os.path.join(save_dir, f"{i}.png")
            txt_file = os.path.join(save_dir, f"{i}.txt")
            zipf.write(img_file, os.path.basename(img_file))
            zipf.write(txt_file, os.path.basename(txt_file))
            os.remove(img_file)
            os.remove(txt_file)

In [15]:
iteration = 1
archive_count = 1
try:
    while True:
        prompt = random.choice(objects)
        prompt += ", isometric,claymorphism,3d render,icon,web icon,clean background,3d figure,minimalistic,simple"

        image = pipe(prompt=prompt, num_inference_steps=5, guidance_scale=0.3).images[0]

        save_image_and_prompt(image, prompt, iteration)

        if iteration % 500 == 0:
            archive_files(iteration, archive_count)
            archive_count += 1
        if iteration % 50 == 0:
            print("-"*20)
            print(iteration)
            print("-"*20)

        iteration += 1
except KeyboardInterrupt:
    print("Генерация остановлена")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Генерация остановлена


In [14]:
folder_path = '/kaggle/working/'
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) or os.path.islink(file_path):
        os.unlink(file_path)
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)
